In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from AnimalShelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "aac1234"
shelter = AnimalShelter(username, password)

# filter queries
water = {
    "breed": {
        "$in" : ["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]
    },
    "sex_upon_outcome" : "Intact Female",
    "age_upon_outcome_in_weeks" : {
        "$gte" : 26,
        "$lte" : 156
    }
}
mountain_wilderness = {
    "breed": {
        "$in" : ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
    },
    "sex_upon_outcome" : "Intact Male",
    "age_upon_outcome_in_weeks" : {
        "$gte" : 26,
        "$lte" : 156
    }
}
disaster_individual = {
    "breed": {
        "$in" : ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
    },
    "sex_upon_outcome" : "Intact Male",
    "age_upon_outcome_in_weeks" : {
        "$gte" : 20,
        "$lte" : 300
    }
}
reset = {}

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read(reset))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project 2')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(
        html.Div([
            html.H2("Application Programmer: John W. Costello III"),
            html.A(
                [html.Img(src="GraziosoSalvareLogo.png", alt="Grazioso Salvare Logo", style={'width': '100px', 'height': '100px'})],
                href='https://www.snhu.edu',
                target='_blank',
            ),
        ])
    ),
    html.Hr(),
    dcc.RadioItems(
        id='radio-buttons',
        options=[
            {'label': 'Water', 'value': 'water'},
            {'label': 'Mountain or Wilderness', 'value': 'mountain_wilderness'},
            {'label': 'Disaster or Individual Tracking', 'value': 'disaster_individual'},
            {'label': 'Reset', 'value': 'reset'}
        ],
        value='reset',  # Set the default selected value
        labelStyle={'display': 'inline-block'}  # Display labels in a horizontal line
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        selected_rows=[0],
        page_size=5,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        column_selectable='single'
    ),
    html.Br(),
    html.Hr(),
    html.Div(
            id='map-id',
            style={
                "width": "45%",
                "display": "inline-block"
            },
    ),
    html.Div(
        [
            dcc.Graph(id='pie-chart-id')
        ],
        style={"width": "45%",
                "display": "inline-block"
              },
    )
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is not None:
        return [{
            'if': { 'column_id': i },
            'background_color': '#D2F3FF'
        } for i in selected_columns]
    else:
        return [{}]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '100%', 'height': '500px'},
        center=[30.75,-97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and popup
        # Column 13 and 14 define the grid-coordinates for 
        # the map
        # Column 4 defines the breed for the animal
        # Column 9 defines the name of the animal
        dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
            children=[
            dl.Tooltip(dff.iloc[row,4]),
            dl.Popup([
                html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]

# Update pie chart the same way that the map is updated
@app.callback(
    Output('pie-chart-id', "figure"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_pie_chart(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    percentage_dff = dff['breed'].value_counts(normalize=True) * 100

    return px.pie(
        names=percentage_dff.index,
        values=percentage_dff.values,
        title='Percentage of Each Breed'
    )

# Update map and chart based on radio button selection
@app.callback(
    Output('datatable-id', "data"),
    [Input('radio-buttons', 'value')]
)
def update_filter(selected_option):
    if (selected_option == "water"):
        df = pd.DataFrame.from_records(shelter.read(water))
    elif (selected_option == "mountain_wilderness"):
        df = pd.DataFrame.from_records(shelter.read(mountain_wilderness))
    elif (selected_option == "disaster_individual"):
        df = pd.DataFrame.from_records(shelter.read(disaster_individual))
    else:
        df = pd.DataFrame.from_records(shelter.read({}))
    
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

app.run_server(debug=True)

Dash app running on http://127.0.0.1:12574/
